In [94]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [95]:
def data_load():
    data=pd.read_excel('./use.xlsx')
    train_data,test_data=train_test_split(data,test_size=0.2, random_state=1314)
    train_data=train_data.drop(['Unnamed: 0'],axis=1)
    test_data=test_data.drop(['Unnamed: 0'],axis=1)
    return train_data,test_data

In [96]:
train_data,test_data=data_load()

In [97]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import pandas as pd

class RegressionNet(nn.Module):
    def __init__(self, input_dim,hidden_dim, output_dim):
        super(RegressionNet, self).__init__()
        self.hidden_layer = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.hidden_layer(x)
        x = self.activation(x)
        x = self.output_layer(x)
        return x

# 读入数据集
data_df = train_data

# 划分特征和标签
X = data_df.iloc[:, :-1].values
y = data_df.iloc[:, -1].values.reshape(-1, 1) # 将标签转换为2D张量

# 使用 DataLoader 准备数据
dataset = data.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dataloader = data.DataLoader(dataset, batch_size=16, shuffle=True)

# 定义模型
model = RegressionNet(input_dim=41, hidden_dim=64, output_dim=1)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
# for i in range(10):
EPOCHS = 1000
for epoch in range(EPOCHS):
    for i, (X_batch, y_batch) in enumerate(dataloader, 1):
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    # 打印损失
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
print('训练完成')

# 使用测试集进行预测并计算损失
test_df = test_data
X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values.reshape(-1, 1)

test_losses = []
with torch.no_grad():
    model.eval()
    y_pred_test = model(torch.Tensor(X_test))
    test_loss = criterion(y_pred_test, torch.Tensor(y_test))
    test_losses.append(test_loss.item())
    print(f'Test Loss: {test_loss.item():.4f}')

Epoch 0, Loss: 1.5869
Epoch 100, Loss: 0.0114
Epoch 200, Loss: 0.1088
Epoch 300, Loss: 0.1369
Epoch 400, Loss: 0.0962
Epoch 500, Loss: 0.0150
Epoch 600, Loss: 0.0636
Epoch 700, Loss: 0.0171
Epoch 800, Loss: 0.0103
Epoch 900, Loss: 0.0253
训练完成
Test Loss: 0.0232


In [98]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
print('测试集得分为：', r2_score(y_test,y_pred_test),pow(mean_squared_error(y_test,y_pred_test),0.5))

测试集得分为： 0.11801688664660226 0.15245958949776445
